In [2]:
import pandas as pd

In [3]:
import duckdb
import pandas as pd
import sqlalchemy

In [4]:
import os
os.chdir('../')

In [5]:
os.getcwd()

'/home/antonio/git/trending-github'

In [6]:
%load_ext sql

In [7]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [8]:
%sql duckdb:///repos.duckdb

In [7]:
%sql df_old << select * from read_parquet('./db/python.parquet/*/*.parquet')

Returning data to local variable df_old


In [ ]:
%sql df_new << select * from read_parquet('/tmp/repos.parquet')

Returning data to local variable df_new


In [ ]:
df_old_new = df_old[df_new.columns]

In [ ]:
%sql df_old_new << select * from df_old_new where insert_date = (select min(insert_date) from df_old_new)

Returning data to local variable df_old_new


In [16]:
%%sql 
create table if not exists repos (
      insert_date date,
      full_name varchar,
      html_url varchar,
      url varchar,
      language varchar,
      homepage varchar,
      description varchar,
      forks_count float,
      stargazers_count float,
      topics varchar[],
      primary key(insert_date, full_name)
    )

""


In [28]:
%%sql 
create or replace temp table t as 
with tmp as ( 
    select *, row_number() over(partition by full_name, insert_date) as rn
    from df_old_new
)
select * from tmp where rn <= 1

,Count
0,2751


In [29]:
%sql alter table t drop rn

""


In [32]:
%sql insert into repos select * from t

,Count
0,2751


In [9]:
%sql select count(*) from repos

,count_star()
0,2751


# Top Repos

In [13]:
%%sql
select full_name,
       html_url,
       homepage,
       insert_date,
       stargazers_count,
       forks_count,
       topics
from read_parquet('./db/python.parquet/*/*.parquet')
where insert_date = (select max(insert_date) from read_parquet('./db/python.parquet/*/*.parquet'))
-- and stargazers_count < 5000
order by stargazers_count desc
limit 30

,full_name,html_url,homepage,insert_date,stargazers_count,forks_count,topics
0,public-apis/public-apis,https://github.com/public-apis/public-apis,http://public-apis.org,2023-01-06,223047,25471,"[api, apis, dataset, development, free, list, ..."
1,donnemartin/system-design-primer,https://github.com/donnemartin/system-design-p...,,2023-01-06,207347,37193,"[design, design-patterns, design-system, devel..."
2,vinta/awesome-python,https://github.com/vinta/awesome-python,https://awesome-python.com/,2023-01-06,152374,22397,"[awesome, collections, python, python-framewor..."
3,TheAlgorithms/Python,https://github.com/TheAlgorithms/Python,https://the-algorithms.com/,2023-01-06,150702,38654,"[algorithm, algorithm-competitions, algorithms..."
4,jackfrued/Python-100-Days,https://github.com/jackfrued/Python-100-Days,,2023-01-06,129303,47852,[]
5,ytdl-org/youtube-dl,https://github.com/ytdl-org/youtube-dl,http://ytdl-org.github.io/youtube-dl/,2023-01-06,116362,8506,[]
6,huggingface/transformers,https://github.com/huggingface/transformers,https://huggingface.co/transformers,2023-01-06,77457,17454,"[bert, deep-learning, flax, hacktoberfest, jax..."
7,nvbn/thefuck,https://github.com/nvbn/thefuck,,2023-01-06,75070,3305,"[python, shell]"
8,tensorflow/models,https://github.com/tensorflow/models,,2023-01-06,74966,46125,[]
9,django/django,https://github.com/django/django,https://www.djangoproject.com/,2023-01-06,67999,28405,"[apps, django, framework, models, orm, python,..."


# Top 10 topics

In [14]:
%%sql
with t as (
select full_name,
       html_url,
       homepage,
       insert_date,
       stargazers_count,
       forks_count,
       unnest(topics) as topic
from read_parquet('./db/python.parquet/*/*.parquet')
where insert_date = (select max(insert_date) from read_parquet('./db/python.parquet/*/*.parquet'))
)
select topic, count(distinct full_name) num_repos
from t
group by topic
order by count(distinct full_name) desc
limit 15

,topic,num_repos
0,python,860
1,deep-learning,275
2,machine-learning,257
3,pytorch,229
4,hacktoberfest,149
5,tensorflow,137
6,python3,114
7,data-science,90
8,computer-vision,73
9,nlp,67


# top 15 repos per topic

In [15]:
%%sql
with t as (
select full_name,
       html_url,
       homepage,
       insert_date,
       stargazers_count,
       forks_count,
       unnest(topics) as topic
from read_parquet('./db/python.parquet/*/*.parquet')
where insert_date = (select max(insert_date) from read_parquet('./db/python.parquet/*/*.parquet'))
)

,top10_topic as (
select topic, count(distinct full_name) num_repos
from t
group by topic
order by count(distinct full_name) desc
limit 10
)
,top10_join as (
    select t.*
    from t join top10_topic on t.topic = top10_topic.topic
)
,final as (
select *, row_number() over(partition by topic order by stargazers_count desc) as rn
from top10_join)

select *
from final
where rn <= 15

,full_name,html_url,homepage,insert_date,stargazers_count,forks_count,topic,rn
0,d2l-ai/d2l-zh,https://github.com/d2l-ai/d2l-zh,http://zh.d2l.ai,2023-01-06,37080,8323,computer-vision,1
1,junyanz/pytorch-CycleGAN-and-pix2pix,https://github.com/junyanz/pytorch-CycleGAN-an...,,2023-01-06,18994,5628,computer-vision,2
2,d2l-ai/d2l-en,https://github.com/d2l-ai/d2l-en,https://D2L.ai,2023-01-06,16045,3484,computer-vision,3
3,huggingface/datasets,https://github.com/huggingface/datasets,https://huggingface.co/docs/datasets,2023-01-06,14999,1957,computer-vision,4
4,pytorch/vision,https://github.com/pytorch/vision,https://pytorch.org/vision,2023-01-06,13076,6438,computer-vision,5
...,...,...,...,...,...,...,...,...
145,onnx/onnx,https://github.com/onnx/onnx,https://onnx.ai/,2023-01-06,13888,3251,tensorflow,11
146,horovod/horovod,https://github.com/horovod/horovod,http://horovod.ai,2023-01-06,12928,2146,tensorflow,12
147,davidsandberg/facenet,https://github.com/davidsandberg/facenet,,2023-01-06,12834,4782,tensorflow,13
148,microsoft/nni,https://github.com/microsoft/nni,https://nni.readthedocs.io,2023-01-06,12367,1735,tensorflow,14


In [17]:
%%sql
select full_name,
       html_url,
       homepage,
       insert_date,
       stargazers_count,
       forks_count
       -- unnest(topics) as topic
from read_parquet('./db/python.parquet/*/*.parquet')
where full_name like '%instrument%'

,full_name,html_url,homepage,insert_date,stargazers_count,forks_count
0,joerick/pyinstrument,https://github.com/joerick/pyinstrument,https://pyinstrument.readthedocs.io/,2023-01-06,4998,217
1,joerick/pyinstrument,https://github.com/joerick/pyinstrument,https://pyinstrument.readthedocs.io/,2023-01-04,4994,217


# Trend

In [36]:
26093/25409

1.026919595418946

In [65]:
%%sql
with final as (
select full_name,
       html_url,
       homepage,
       topics,
       list(insert_date) over(partition by full_name order by insert_date) as insert_date,
       list(stargazers_count) over(partition by full_name order by insert_date) as stars,
       stargazers_count,
        --list(forks_count) over(partition by full_name order by insert_date) as forks,
       lag(stargazers_count) over(partition by full_name order by insert_date) as prev
       -- unnest(topics) as topic
from read_parquet('./db/python.parquet/*/*.parquet')
)
select *,
    (stargazers_count/cast(prev as float))-1 perc
from final
where prev is not null
order by (stargazers_count/cast(prev as float))-1 desc


,full_name,html_url,homepage,topics,insert_date,stars,stargazers_count,prev,perc
0,bigscience-workshop/petals,https://github.com/bigscience-workshop/petals,https://petals.ml,"[bloom, deep-learning, distributed-systems, la...","[2023-01-04, 2023-01-06]","[1864, 2057]",2057,1864,0.103541
1,karpathy/minGPT,https://github.com/karpathy/minGPT,None,[],"[2023-01-04, 2023-01-06]","[9424, 9949]",9949,9424,0.055709
2,nonebot/nonebot2,https://github.com/nonebot/nonebot2,https://v2.nonebot.dev,"[bot, chatbot, cqhttp, dingtalk-robot, fastapi...","[2023-01-04, 2023-01-06]","[3062, 3225]",3225,3062,0.053233
3,hpcaitech/ColossalAI,https://github.com/hpcaitech/ColossalAI,https://www.colossalai.org/,"[ai, big-model, data-parallelism, deep-learnin...","[2023-01-04, 2023-01-06]","[7331, 7584]",7584,7331,0.034511
4,openai/openai-cookbook,https://github.com/openai/openai-cookbook,None,[],"[2023-01-04, 2023-01-06]","[3675, 3784]",3784,3675,0.029660
...,...,...,...,...,...,...,...,...,...
2712,fossasia/mew,https://github.com/fossasia/mew,,[],"[2023-01-04, 2023-01-06]","[1484, 1482]",1482,1484,-0.001348
2713,fossasia/open-event-legacy,https://github.com/fossasia/open-event-legacy,,[],"[2023-01-04, 2023-01-06]","[1478, 1476]",1476,1478,-0.001353
2714,yoda-pa/yoda,https://github.com/yoda-pa/yoda,https://manparvesh.com/project/yoda/,"[chat, chatbot, cli, devtools, diary, dude, le...","[2023-01-04, 2023-01-06]","[700, 699]",699,700,-0.001429
2715,benedekrozemberczki/ClusterGCN,https://github.com/benedekrozemberczki/ClusterGCN,,"[clustering, community-detection, deep-learnin...","[2023-01-04, 2023-01-06]","[697, 696]",696,697,-0.001435


In [67]:
%%sql
select full_name,
      html_url,
      homepage,
      insert_date,
      stargazers_count,
      forks_count,
      topics
    from read_parquet('./db/python.parquet/*/*.parquet')
    where insert_date = (select max(insert_date) from read_parquet('./db/python.parquet/*/*.parquet'))
      and list_has(topics, 'cli')
    order by stargazers_count desc
    limit 50

,full_name,html_url,homepage,insert_date,stargazers_count,forks_count,topics
0,sherlock-project/sherlock,https://github.com/sherlock-project/sherlock,http://sherlock-project.github.io,2023-01-06,38289,4525,"[cli, hacktoberfest, hacktoberfest2022, inform..."
1,chubin/cheat.sh,https://github.com/chubin/cheat.sh,https://cheat.sh/,2023-01-06,34349,1650,"[cheatsheet, cli, command-line, curl, document..."
2,httpie/httpie,https://github.com/httpie/httpie,https://httpie.io,2023-01-06,25560,3728,"[api, api-client, api-testing, cli, client, cu..."
3,tqdm/tqdm,https://github.com/tqdm/tqdm,https://tqdm.github.io,2023-01-06,23701,1202,"[cli, closember, console, discord, gui, jupyte..."
4,google/python-fire,https://github.com/google/python-fire,,2023-01-06,23619,1358,"[cli, python]"
5,Textualize/textual,https://github.com/Textualize/textual,https://textual.textualize.io/,2023-01-06,17078,492,"[cli, framework, python, rich, terminal, tui]"
6,pallets/click,https://github.com/pallets/click,https://click.palletsprojects.com,2023-01-06,13346,1308,"[cli, click, pallets, python]"
7,beetbox/beets,https://github.com/beetbox/beets,http://beets.io/,2023-01-06,11354,1763,"[cli, hacktoberfest, music, music-library, mus..."
8,asciinema/asciinema,https://github.com/asciinema/asciinema,https://asciinema.org,2023-01-06,11108,853,"[asciicast, cli, recorder, recording, terminal..."
9,tiangolo/typer,https://github.com/tiangolo/typer,https://typer.tiangolo.com/,2023-01-06,10140,402,"[cli, click, python, python3, shell, terminal,..."


In [71]:
%sql df << select * from read_parquet('./db/python.parquet/*/*.parquet')

Returning data to local variable df


In [83]:
df.head(3)

,id,node_id,name,full_name,private,owner,html_url,description,fork,url,...,is_template,web_commit_signoff_required,topics,visibility,forks,open_issues,watchers,default_branch,score,insert_date
0,54346799,MDEwOlJlcG9zaXRvcnk1NDM0Njc5OQ==,public-apis,public-apis/public-apis,False,{'avatar_url': 'https://avatars.githubusercont...,https://github.com/public-apis/public-apis,A collective list of free APIs,False,https://api.github.com/repos/public-apis/publi...,...,False,False,"[api, apis, dataset, development, free, list, ...",public,25471,115,223047,master,1.0,2023-01-06
1,83222441,MDEwOlJlcG9zaXRvcnk4MzIyMjQ0MQ==,system-design-primer,donnemartin/system-design-primer,False,{'avatar_url': 'https://avatars.githubusercont...,https://github.com/donnemartin/system-design-p...,Learn how to design large-scale systems. Prep ...,False,https://api.github.com/repos/donnemartin/syste...,...,False,False,"[design, design-patterns, design-system, devel...",public,37193,343,207347,master,1.0,2023-01-06
2,21289110,MDEwOlJlcG9zaXRvcnkyMTI4OTExMA==,awesome-python,vinta/awesome-python,False,{'avatar_url': 'https://avatars.githubusercont...,https://github.com/vinta/awesome-python,"A curated list of awesome Python frameworks, l...",False,https://api.github.com/repos/vinta/awesome-python,...,False,False,"[awesome, collections, python, python-framewor...",public,22397,312,152374,master,1.0,2023-01-06


In [84]:
%sql CREATE TABLE new_tbl AS SELECT * FROM read_parquet('./db/python.parquet/*/*.parquet')


,Count
0,5518


In [85]:
df.to_parquet('/tmp/tmp.parquet')

In [86]:
%sql insert into new_tbl select * from '/tmp/tmp.parquet'

,Count
0,5518


In [88]:
%sql select count(*) from new_tbl

,count_star()
0,11036
